In [60]:
import cv2
import numpy as np
import os

# --- Parameters ---
PROCESSING_WIDTH = 480
PATCH_SIZE = 9
OMEGA = 0.95
T0 = 0.1
ALPHA = 0.4

In [61]:
def get_dark_channel(frame, patch_size):
    """Calculates the dark channel of an image."""
    min_channel = np.min(frame, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    dark_channel = cv2.erode(min_channel, kernel)
    return dark_channel

def get_airlight(frame, dark_channel):
    """Estimates the atmospheric airlight."""
    h, w = dark_channel.shape
    num_pixels = h * w
    num_brightest = int(num_pixels * 0.001)
    flat_dark = dark_channel.flatten()
    flat_frame = frame.reshape(num_pixels, 3)
    indices = np.argsort(flat_dark)[-num_brightest:]
    brightest_pixels = flat_frame[indices]
    airlight = np.mean(brightest_pixels, axis=0)
    return airlight.astype(np.uint8)

def get_transmission_map(frame, airlight, patch_size, omega):
    """Estimates the transmission map."""
    airlight_safe = np.maximum(airlight.astype(np.float64), 1)
    normalized_frame = frame.astype(np.float64) / airlight_safe
    dark_channel_norm = get_dark_channel(normalized_frame, patch_size)
    transmission = 1 - omega * dark_channel_norm
    return transmission

def refine_transmission_map(frame, transmission_map):
    """Refines the transmission map using the fast Guided Filter."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    transmission_map_float = transmission_map.astype(np.float32)
    refined_t = cv2.ximgproc.guidedFilter(guide=gray_frame, src=transmission_map_float, radius=50, eps=0.001)
    return refined_t

def restore_image(frame, airlight, transmission_map, t0):
    """Restores the haze-free image."""
    t_clipped = np.maximum(transmission_map, t0)
    if t_clipped.ndim == 2:
        t_reshaped = cv2.cvtColor(t_clipped, cv2.COLOR_GRAY2BGR)
    else:
        t_reshaped = t_clipped
    frame_float = frame.astype(np.float64)
    airlight_float = airlight.astype(np.float64)
    restored = (frame_float - airlight_float) / t_reshaped + airlight_float
    return np.clip(restored, 0, 255).astype(np.uint8)

def apply_clahe(frame):
    """Applies CLAHE for final contrast enhancement."""
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def temporal_smoothing(current_frame, previous_frame, alpha):
    """Applies IIR filter for temporal smoothing between frames."""
    if previous_frame is None:
        return current_frame
    return cv2.addWeighted(current_frame, alpha, previous_frame, 1 - alpha, 0)

In [62]:
def process_video(input_path, output_path):
    """Main function to process the video with optimizations."""
    if not os.path.exists(input_path):
        print(f"Error: Input video not found at '{input_path}'")
        return

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {input_path}")
        return

    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Using a compatible codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (original_width, original_height))

    aspect_ratio = original_height / original_width
    processing_height = int(PROCESSING_WIDTH * aspect_ratio)

    prev_frame_smoothed = None
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}...")

        small_frame = cv2.resize(frame, (PROCESSING_WIDTH, processing_height), interpolation=cv2.INTER_AREA)

        dark_channel = get_dark_channel(small_frame, PATCH_SIZE)
        airlight = get_airlight(small_frame, dark_channel)

        transmission_map = get_transmission_map(small_frame, airlight, PATCH_SIZE, OMEGA)
        refined_t_small = refine_transmission_map(small_frame, transmission_map)

        full_size_t = cv2.resize(refined_t_small, (original_width, original_height), interpolation=cv2.INTER_LINEAR)

        restored_frame = restore_image(frame, airlight, full_size_t, T0)

        enhanced_frame = apply_clahe(restored_frame)

        smoothed_frame = temporal_smoothing(enhanced_frame, prev_frame_smoothed, ALPHA)

        prev_frame_smoothed = smoothed_frame
        out.write(smoothed_frame)

    print(f"\n Finished processing! Video saved to {output_path}")
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [63]:
if __name__ == '__main__':
    input_video_name = 'hazy_input.mp4'
    output_video_name = 'dehazed_output_final_v1.mp4'
    process_video(input_video_name, output_video_name)

Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Processing frame 31...
Processing frame 32...
Processing frame 33...
Processing frame 34...
Processing frame 35...
Processing frame 36...
Processing frame 37...
Processing frame 38...
Processing frame 39...
Processing frame 40...
Processing frame 41...
Processing frame 42...
Processing frame 43...
Processing frame 44.